In [7]:
import torch
import torch.nn as nn #contains different classess that help you build neural network models. like activation fun forward
import torchvision #for datasets
import torchvision.transforms as transforms #used for image transforms to tensor
import matplotlib.pyplot as plt #to plot

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_size=784
hi_size=100
num_classes=10
num_ep=100
batch_size=512
learning_rate=0.005
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)#downloads the data 
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
#Combines a dataset and a sampler, and provides an iterable over the given datas
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)
print(len(train_loader))
examples= iter(train_loader) # returns an iterator for the given object
samples,labels=examples.next()
print(samples.shape,labels.shape)

In [ ]:
class NeuralNet(nn.Module): #class for making forward prob
  def __init__(self,in_size,hi_size,num_classes):
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(in_size,hi_size)
    self.relu= nn.ReLU()
    self.l2=nn.Linear(hi_size,num_classes)
  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out) #don't use sigmoid as it comes in  cross entropy loss
    return out

model= NeuralNet(in_size,hi_size,num_classes)

criterion=nn.CrossEntropyLoss() #computes loss

optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
#Optimizers are algorithms or methods used to change the attributes of the neural network such as weights and learning rate to reduce the losses
#training loop
train_losses = []
train_counter = []
test_losses = []
n_steps=len(train_loader)
for epoch in range(num_ep):#will loop over epochs
  for i,(images,labels) in enumerate(train_loader): #enumerate loops over the given data and gives labels i mean changes them after every iter
    images=images.reshape(-1,28*28).to(device) #before it was 100,1,28,28 now 100,784
    labels=labels.to(device)
    #forward
    outputs=model(images)
    loss=criterion(outputs,labels)
    train_losses.append(loss.item())

    #backward
    optimizer.zero_grad()#empty grad
    loss.backward()#does backprop
    optimizer.step()#upates the parameters for us
    if (i+1) % 100 == 0:
      print(f'Train Epoch: {epoch+1}/{num_ep}, step{i+1}/{n_steps}, loss ={loss.item():.4f}')



def test(): 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
      images = images.reshape(-1, 28*28)
      outputs = model(images)

      _,predictions = torch.max(outputs, 1) #torch.max function will return the value and the index and we are interested in the actual index
      n_samples += labels.shape[0]
      n_correct += (predictions == labels).sum().item()
  
    acc = 100* n_correct / n_samples
    print(f'Test accuracy = {acc:.4f}')

def train(): 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in train_loader:
      images = images.reshape(-1, 28*28)
      outputs = model(images)

      _,predictions = torch.max(outputs, 1) #torch.max function will return the value and the index and we are interested in the actual index
      n_samples += labels.shape[0]
      n_correct += (predictions == labels).sum().item()
  
    acc = 100* n_correct / n_samples
    print(f'Training accuracy = {acc:.4f}')

train()
test()






          

In [ ]:
plt.plot(train_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.title("Learning rate 0.001")

In [ ]:
for images, labels in test_loader:
      sample_image = images[0].reshape(-1, 28*28)
      output = model(sample_image)
      sample_label=labels[0]
      _,prediction = torch.max(output, 1)
      print(prediction)
      print(sample_label)

